In [4]:
%matplotlib inline
import quadkey
import os
import sys
import operator
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import sparse
from sklearn import model_selection, preprocessing, ensemble
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import OneHotEncoder
import quadkey

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
import xgboost as xgb

In [6]:
data_path = "input/"
train_file = data_path + "train.json"
test_file = data_path + "test.json"
train_df = pd.read_json(train_file)
test_df = pd.read_json(test_file)
print(train_df.shape)
print(test_df.shape)

(49352, 15)
(74659, 14)


In [6]:
train_df.head()

,bathrooms,bedrooms,building_id,created,description,display_address,features,interest_level,latitude,listing_id,longitude,manager_id,photos,price,street_address
10,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,[],medium,40.7145,7211212,-73.9425,5ba989232d0489da1b5f2c45f6688adc,[https://photos.renthop.com/2/7211212_1ed4542e...,3000,792 Metropolitan Avenue
10000,1.0,2,c5c8a357cba207596b04d1afd1e4f130,2016-06-12 12:19:27,,Columbus Avenue,"[Doorman, Elevator, Fitness Center, Cats Allow...",low,40.7947,7150865,-73.9667,7533621a882f71e25173b27e3139d83d,[https://photos.renthop.com/2/7150865_be3306c5...,5465,808 Columbus Avenue
100004,1.0,1,c3ba40552e2120b0acfc3cb5730bb2aa,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,"[Laundry In Building, Dishwasher, Hardwood Flo...",high,40.7388,6887163,-74.0018,d9039c43983f6e564b1482b273bd7b01,[https://photos.renthop.com/2/6887163_de85c427...,2850,241 W 13 Street
100007,1.0,1,28d9ad350afeaab8027513a3e52ac8d5,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,"[Hardwood Floors, No Fee]",low,40.7539,6888711,-73.9677,1067e078446a7897d2da493d2f741316,[https://photos.renthop.com/2/6888711_6e660cee...,3275,333 East 49th Street
100013,1.0,4,0,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,[Pre-War],low,40.8241,6934781,-73.9493,98e13ad4b495b9613cef886d79a6291f,[https://photos.renthop.com/2/6934781_1fa4b41a...,3350,500 West 143rd Street


In [7]:
vectorizer = DictVectorizer( sparse = False )
vec_x_cat_train = pd.get_dummies(train_df['manager_id'])
#vec_x_cat_test = vectorizer.transform( train_df['interest_level'] ) 
#train_df['interest_level'].map({'high':0, 'medium':1, 'low':2})

In [8]:
train_df.columns

Index([u'bathrooms', u'bedrooms', u'building_id', u'created', u'description',
       u'display_address', u'features', u'interest_level', u'latitude',
       u'listing_id', u'longitude', u'manager_id', u'photos', u'price',
       u'street_address'],
      dtype='object')

In [7]:
def feature_preprocessing(df, if_train=True):
    
    target_num_map = {'high':0, 'medium':1, 'low':2}
    features_to_use  = ["bathrooms", "bedrooms", "latitude", "longitude",  "price" ,
                        "diff_price",'diff_bathrooms','diff_bedrooms',
                        'ratio_price',
                       'building_price','building_bathrooms', 'building_bedrooms',
                       'diff_mean_of_price', 'diff_mean_of_bedrooms', 'diff_mean_of_bathrooms',
                       'diff_bathrooms_quadkey_15', 'diff_bedrooms_quadkey_15', 'diff_price_quadkey_15',
                        'ratio_price_quadkey_15',
                       'diff_bathrooms_quadkey_13', 'diff_bedrooms_quadkey_13', 'diff_price_quadkey_13',
                       'ratio_price_quadkey_13',
                       'price_per_bed', 'price_per_bath','price_per_room',
                       'building_price_per_room', 'diff_price_per_rooms',
                        'total_room','diff_room','ratio_room']
    
    df = df[df.columns]
    # datetime
    df['ts'] = pd.to_datetime(df['created'])
    df['created_year'] = df['ts'].dt.year
    df['created_month'] = df['ts'].dt.month
    df['created_day'] = df['ts'].dt.day
    df['created_hour'] = df['ts'].dt.hour
    
    #room
    df['total_room'] = df['bathrooms'] + df['bedrooms'] 
    df['diff_room'] = df['bathrooms'] - df['bedrooms'] 
    df['ratio_room'] = df['bathrooms'] / df['bedrooms']
    
    
    #len
    df['features_cnt'] = df['features'].apply(len)
    df['photos_cnt'] = df['photos'].apply(len)
    df['desc_cnt'] = df['description'].apply(len)
    df['address_cnt'] = df['display_address'].apply(len)
    df['street_address_cnt'] = df['street_address'].apply(len)
    
    #price per 
    
    df['price_per_bed']  = df['price'] / df['bedrooms']    
    df['price_per_bath'] = df['price'] / df['bathrooms']
    df['price_per_room'] = df['price'] / (df['bathrooms'] + df['bedrooms'] )
    
    # group by building id
    mean_of_building = df.groupby('building_id')['bathrooms','bedrooms','price'].mean()
    mean_of_building.columns = ['building_bathrooms','building_bedrooms','building_price']
    df = pd.merge(df, mean_of_building, left_on = df.building_id, right_index=True, how='left')
    df['building_price'].fillna(df['price'])
    df['building_bedrooms'].fillna(df['bedrooms'])    
    df['building_bathrooms'].fillna(df['bathrooms'])
    
    df['building_price_per_room'] = df['building_price']/(df['building_bedrooms']+df['building_bathrooms'])
    
    df['diff_bathrooms'] = df['bathrooms'] - df['building_bathrooms']
    df['diff_bedrooms'] = df['bedrooms'] - df['building_bedrooms']
    df['diff_price'] = df['price'] - df['building_price']
    df['diff_price_per_rooms'] = df['price_per_room'] - df['building_price_per_room']
    
    df['ratio_price'] = df['price'] / df['building_price']
    
    #avg price
    df['diff_mean_of_price'] = df['price'] - df['price'].mean()
    df['diff_mean_of_bedrooms'] = df['bedrooms'] - df['bedrooms'].mean()
    df['diff_mean_of_bathrooms'] = df['bathrooms'] - df['bathrooms'].mean()
    
    #return df
    
    # add quadkey 
    #df['quadkey_15'] = quadkey.from_geo((df['latitude'], df['longitude']), 15)
    #df['quadkey_14'] = quadkey.from_geo((df['latitude'], df['longitude']), 14)
    #df['quadkey_13'] = quadkey.from_geo((df['latitude'], df['longitude']), 13)
    
    df['quadkey_15'] = df.apply(lambda x: quadkey.from_geo((x['latitude'], x['longitude']), 15).key, axis=1)
    #df['quadkey_14'] = df.apply(lambda x: quadkey.from_geo((x['latitude'], x['longitude']), 14),axis=1)
    df['quadkey_13'] = df.apply(lambda x: quadkey.from_geo((x['latitude'], x['longitude']), 13).key,axis=1)
    
    
    #group by quadkey
    mean_of_quadkey_15 = df.groupby('quadkey_15')['bathrooms','bedrooms','price'].mean()
    mean_of_quadkey_15.columns = ['quadkey_15_bathrooms','quadkey_15_bedrooms','quadkey_15_price']
    print mean_of_quadkey_15.head()
    df = pd.merge(df, mean_of_quadkey_15, left_on = df.quadkey_15, right_index=True, how='left')
    df['quadkey_15_price'].fillna(df['price'])
    df['quadkey_15_bedrooms'].fillna(df['bedrooms'])    
    df['quadkey_15_bathrooms'].fillna(df['bathrooms'])
    
    df['diff_bathrooms_quadkey_15'] = df['bathrooms'] - df['quadkey_15_bathrooms']
    df['diff_bedrooms_quadkey_15'] = df['bedrooms'] - df['quadkey_15_bedrooms']
    df['diff_price_quadkey_15'] = df['price'] - df['quadkey_15_price']
    df['ratio_price_quadkey_15'] = df['price'] / df['quadkey_15_price']
    
    
    mean_of_quadkey_13 = df.groupby('quadkey_13')['bathrooms','bedrooms','price'].mean()
    mean_of_quadkey_13.columns = ['quadkey_13_bathrooms','quadkey_13_bedrooms','quadkey_13_price']
    df = pd.merge(df, mean_of_quadkey_13, left_on = df.quadkey_13, right_index=True, how='left')
    df['quadkey_13_price'].fillna(df['price'])
    df['quadkey_13_bedrooms'].fillna(df['bedrooms'])    
    df['quadkey_13_bathrooms'].fillna(df['bathrooms'])
    
    df['diff_bathrooms_quadkey_13'] = df['bathrooms'] - df['quadkey_13_bathrooms']
    df['diff_bedrooms_quadkey_13'] = df['bedrooms'] - df['quadkey_13_bedrooms']
    df['diff_price_quadkey_13'] = df['price'] - df['quadkey_13_price']
    
    df['ratio_price_quadkey_13'] = df['price'] / df['quadkey_13_price']
    
    #return df
    
    #features
    df['features'] = df["features"].apply(lambda x: 'Ringcentral ' + " ".join(["_".join(i.split(" ")) for i in x]))
    df['features'] = df['features'].str.replace("garden/patio","garden")
    df['features'] = df['features'].str.replace("patio","garden")
    df['features'] = df['features'].str.replace("residents_garden","garden")
    df['features'] = df['features'].str.replace("common_garden","garden")
    df['features'] = df['features'].str.replace("wifi_access","wifi")
    df['features'] = df['features'].str.replace("24/7","24")
    df['features'] = df['features'].str.replace("24-hour","24")
    df['features'] = df['features'].str.replace("24hr","24")
    df['features'] = df['features'].str.replace("concierge","doorman")
    df['features'] = df['features'].str.replace("ft_doorman","doorman")
    df['features'] = df['features'].str.replace("24_doorman","doorman")
    df['features'] = df['features'].str.replace("24_hr_doorman","doorman")
    df['features'] = df['features'].str.replace("doorman_service","doorman")
    df['features'] = df['features'].str.replace("full-time_doorman","doorman")
    
    tfidf = CountVectorizer(stop_words='english' , max_features=100, min_df=1)
    tr_sparse = tfidf.fit_transform(df["features"])
    print 'tr_sparse' + str(tr_sparse.shape)
    
    #description
    tr_sparse_desc = tfidf.fit_transform(df["description"])
    print 'tr_sparse_desc' + str(tr_sparse_desc.shape)
    
    features_to_use.extend(['created_year', 'created_month', 'created_day', 'created_hour','features_cnt','photos_cnt','desc_cnt'])
    train_x = df[features_to_use]
    print 'cloumns: {0}'.format(train_x.columns)
    
    #manager id 
    
    
    
    
    train_x = sparse.hstack([train_x, tr_sparse, tr_sparse_desc]).tocsr()
    #train_x = sparse.csr_matrix(train_x.values)

    if if_train:
        train_y = pd.DataFrame(df['interest_level'])
        train_y = np.array(train_y['interest_level'].apply(lambda x: target_num_map[x]))
        print 'train_x: %s; train_y %s' % (str(train_x.shape), str(train_y.shape))
        return train_x, train_y
    else:
        print 'train_x: %s' % (str(train_x.shape))
        return train_x

In [8]:
def prediction(test_df, model):
    target_num_map = {'high':0, 'medium':1, 'low':2}
    test_df_1 = feature_preprocessing(test_df, False)
    preds, model = runXGB(train_X, train_y, test_X, num_rounds=400)
    out_df = pd.DataFrame(preds)
    out_df.columns = ["high", "medium", "low"]
    out_df["listing_id"] = test_df.listing_id.values

    return sub

In [9]:
def add_submit_result( submit_result, cv_scores, score):
    submit_result[str(reduce(lambda x,y: x + y, cv_scores)/len(cv_scores))] = score
    return  submit_result

In [10]:
train_x, train_y = feature_preprocessing(train_df)

                 quadkey_15_bathrooms  quadkey_15_bedrooms  quadkey_15_price
quadkey_15                                                                  
021333011032130                   3.0                  3.0           13875.0
023012311310010                   1.0                  1.0            2425.0
030222231211021                   1.0                  0.0            2000.0
030232101222321                   1.0                  0.0            1350.0
030232211000200                   1.0                  2.0            3300.0
tr_sparse(49352, 100)
tr_sparse_desc(49352, 100)
cloumns: Index([                u'bathrooms',                  u'bedrooms',
                        u'latitude',                 u'longitude',
                           u'price',                u'diff_price',
                  u'diff_bathrooms',             u'diff_bedrooms',
                     u'ratio_price',            u'building_price',
              u'building_bathrooms',         u'building_bedrooms',
 

In [12]:
def runXGB(param, train_X, train_y, test_X, test_y=None, feature_names=None, num_rounds=2000):
    num_rounds = num_rounds
    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y)
        watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=20, verbose_eval=100)
    else:
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst, xgtrain, num_rounds, verbose_eval=100)

    pred_test_y = model.predict(xgtest)
    return pred_test_y, model


In [356]:
# one time

param = {}
    # fixed parems
param['objective'] = 'multi:softprob'
param['eval_metric'] = "mlogloss"
param['seed'] = 0
param['silent'] = 1 
param['num_class'] = 3
    
    # random
param['eta'] = 0.1   #[0,1]
param['max_depth'] = 6                      #[1,]
param['min_child_weight'] = 0                    #[0,]
param['subsample'] = 0.7                 #(0,1]
param['colsample_bytree'] = 0.7          #(0,1]
    
cv_scores = []
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2016)
fold = 0 
for dev_index, val_index in kf.split(range(train_x.shape[0])):
        print "Fold: %s" % fold
        dev_x, val_x = train_x[dev_index,:], train_x[val_index,:]
        dev_y, val_y = train_y[dev_index], train_y[val_index]

        # Xbgoost parameter 
        val_y_pred, model = runXGB(param, dev_x, dev_y, val_x, val_y)
        cv_scores.append(log_loss(val_y, val_y_pred))

        print(cv_scores)
        fold += 1
    


Fold: 0
[0]	train-mlogloss:1.03481	test-mlogloss:1.03666
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 20 rounds.
[100]	train-mlogloss:0.481314	test-mlogloss:0.566267
[200]	train-mlogloss:0.429064	test-mlogloss:0.555033
[300]	train-mlogloss:0.389567	test-mlogloss:0.550174
[400]	train-mlogloss:0.354959	test-mlogloss:0.54862
Stopping. Best iteration:
[400]	train-mlogloss:0.354959	test-mlogloss:0.54862

[0.54872436555021809]
Fold: 1
[0]	train-mlogloss:1.035	test-mlogloss:1.03636
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 20 rounds.
[100]	train-mlogloss:0.481144	test-mlogloss:0.567363
[200]	train-mlogloss:0.427791	test-mlogloss:0.554664
[300]	train-mlogloss:0.390271	test-mlogloss:0.55089
Stopping. Best iteration:
[346]	train-mlogloss:0.373722	test-mlogloss:0.549773

[0.54872436555021809, 0.550133050695

#### 

In [20]:

training_result = {}

for i in range(100):
    
    print 'round: %s' % i
    param = {}
    # fixed parems
    param['objective'] = 'multi:softprob'
    param['eval_metric'] = "mlogloss"
    param['seed'] = 0
    param['silent'] = 1 
    param['num_class'] = 3
    
    # random
    param['eta'] = np.random.choice([0.01, 0.05, 0.1, 0.5, 1], 1)[0]    #[0,1]
    param['max_depth'] = np.random.choice([2,3,4,5,6,7,8,9,10], 1)[0]                       #[1,]
    param['min_child_weight'] = np.random.choice([0,1,2,3,4,5], 1)[0]                    #[0,]
    param['subsample'] = np.random.choice([0.4,0.5,0.6,0.7,0.8], 1)[0]                 #(0,1]
    param['colsample_bytree'] = np.random.choice([0.4,0.5,0.6,0.7,0.8], 1)[0]          #(0,1]

    
    print 'params: %s' % param
    #init result for this round
    training_result[i] = {}
    training_result[i]['param'] = param
    
    cv_scores = []
    kf = model_selection.KFold(n_splits=2, shuffle=True, random_state=2016)
    fold = 0 
    for dev_index, val_index in kf.split(range(train_x.shape[0])):
            print "Fold: %s" % fold
            dev_x, val_x = train_x[dev_index,:], train_x[val_index,:]
            dev_y, val_y = train_y[dev_index], train_y[val_index]

            # Xbgoost parameter 
            val_y_pred, model = runXGB(param, dev_x, dev_y, val_x, val_y)
            cv_scores.append(log_loss(val_y, val_y_pred))

            print(cv_scores)
            fold += 1
    
    training_result[i]['cv_scores'] = cv_scores
    
        
        

round: 0
params: {'num_class': 3, 'silent': 1, 'eval_metric': 'mlogloss', 'eta': 0.01, 'min_child_weight': 0, 'subsample': 0.40000000000000002, 'seed': 0, 'objective': 'multi:softprob', 'colsample_bytree': 0.5, 'max_depth': 3}
Fold: 0
[0]	train-mlogloss:1.09285	test-mlogloss:1.09292
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 20 rounds.
[100]	train-mlogloss:0.782535	test-mlogloss:0.786932
[200]	train-mlogloss:0.686137	test-mlogloss:0.694426
[300]	train-mlogloss:0.645378	test-mlogloss:0.657036
[400]	train-mlogloss:0.623482	test-mlogloss:0.638247
[500]	train-mlogloss:0.608823	test-mlogloss:0.626335
[600]	train-mlogloss:0.597636	test-mlogloss:0.617753
[700]	train-mlogloss:0.588842	test-mlogloss:0.611415
[800]	train-mlogloss:0.581397	test-mlogloss:0.606418
[900]	train-mlogloss:0.574705	test-mlogloss:0.601972
[1000]	train-mlogloss:0.568832	test-mlogloss:0.598337
[1100]	train-mlogloss:0.563553	tes

KeyboardInterrupt: 

In [137]:
training_result

{0: {'cv_scores': [0.58891432777555008,
   0.59059962393845011,
   0.59504002662790501,
   0.57816091002066994,
   0.5870666042403766],
  'param': {'colsample_bytree': 0.80000000000000004,
   'eta': 0.5,
   'eval_metric': 'mlogloss',
   'max_depth': 6,
   'min_child_weight': 1,
   'num_class': 3,
   'objective': 'multi:softprob',
   'seed': 0,
   'silent': 1,
   'subsample': 0.5}},
 1: {'cv_scores': [0.59185194983836598,
   0.61654965019753549,
   0.60507615376509849,
   0.5925758511877478,
   0.58725648113202444],
  'param': {'colsample_bytree': 0.5,
   'eta': 0.5,
   'eval_metric': 'mlogloss',
   'max_depth': 8,
   'min_child_weight': 0,
   'num_class': 3,
   'objective': 'multi:softprob',
   'seed': 0,
   'silent': 1,
   'subsample': 0.59999999999999998}},
 2: {'cv_scores': [0.56162437424026679,
   0.5663586329296797,
   0.56865498546715931,
   0.55313734497072098,
   0.55607142226158446],
  'param': {'colsample_bytree': 0.59999999999999998,
   'eta': 0.10000000000000001,
   'eval_m

In [13]:
def mixed_model(training_result, train_x, train_y, target_x, num_rounds=500):
    mix_model_df = pd.DataFrame()
    for i in xrange(10):
        print 'round: %s' % i
        preds, model = runXGB(training_result[i]['param'], train_x, train_y, target_x, num_rounds=500)
        preds_df = pd.DataFrame(preds, columns=['pred_high_%s' % i, 'pred_medium_%s' % i, 'pred_low_%s' % i])

        if i == 0:
            mix_model_df = preds_df
        else:
            mix_model_df = pd.concat([mix_model_df, preds_df], axis = 1)
        print 'dataframe: {0}'.format(mix_model_df.shape)  
        
    return mix_model_df

In [22]:
# get preds by each model
mix_model_df = pd.DataFrame()
for i in xrange(10):
    print 'round: %s' % i
    param = {}
    # fixed parems
    param['objective'] = 'multi:softprob'
    param['eval_metric'] = "mlogloss"
    param['seed'] = 0
    param['silent'] = 1 
    param['num_class'] = 3
    
    # random
    param['eta'] = np.random.choice([0.01, 0.05, 0.1, 0.5, 1], 1)[0]    #[0,1]
    param['max_depth'] = np.random.choice([2,3,4,5,6,7,8,9,10], 1)[0]                       #[1,]
    param['min_child_weight'] = np.random.choice([0,1,2,3,4,5], 1)[0]                    #[0,]
    param['subsample'] = np.random.choice([0.4,0.5,0.6,0.7,0.8], 1)[0]                 #(0,1]
    param['colsample_bytree'] = np.random.choice([0.4,0.5,0.6,0.7,0.8], 1)[0]          #(0,1]
    print 'parma: %s' % param
    preds, model = runXGB(param, train_x, train_y, train_x, num_rounds=2000)
    preds_df = pd.DataFrame(preds, columns=['pred_high_%s' % i, 'pred_medium_%s' % i, 'pred_low_%s' % i])
    
    if i == 0:
        mix_model_df = preds_df
    else:
        mix_model_df = pd.concat([mix_model_df, preds_df], axis = 1)

    print 'dataframe: {0}'.format(mix_model_df.shape)

round: 0
parma: {'num_class': 3, 'silent': 1, 'eval_metric': 'mlogloss', 'eta': 0.050000000000000003, 'min_child_weight': 5, 'subsample': 0.59999999999999998, 'seed': 0, 'objective': 'multi:softprob', 'colsample_bytree': 0.5, 'max_depth': 4}
dataframe: (49352, 3)
round: 1
parma: {'num_class': 3, 'silent': 1, 'eval_metric': 'mlogloss', 'eta': 0.10000000000000001, 'min_child_weight': 0, 'subsample': 0.5, 'seed': 0, 'objective': 'multi:softprob', 'colsample_bytree': 0.59999999999999998, 'max_depth': 6}
dataframe: (49352, 6)
round: 2
parma: {'num_class': 3, 'silent': 1, 'eval_metric': 'mlogloss', 'eta': 0.01, 'min_child_weight': 2, 'subsample': 0.5, 'seed': 0, 'objective': 'multi:softprob', 'colsample_bytree': 0.80000000000000004, 'max_depth': 6}
dataframe: (49352, 9)
round: 3
parma: {'num_class': 3, 'silent': 1, 'eval_metric': 'mlogloss', 'eta': 0.050000000000000003, 'min_child_weight': 4, 'subsample': 0.80000000000000004, 'seed': 0, 'objective': 'multi:softprob', 'colsample_bytree': 0.80

In [23]:
mix_model_df.shape

(49352, 30)

In [127]:
# ensenble modeling 

In [140]:
train_mix_model = sparse.csr_matrix(mix_model_df.values)

In [141]:
ensenble_training_result = {}

for i in range(10):
    
    print 'round: %s' % i
    param = {}
    # fixed parems
    param['objective'] = 'multi:softprob'
    param['eval_metric'] = "mlogloss"
    param['seed'] = 0
    param['silent'] = 1 
    param['num_class'] = 3
    
    # random
    param['eta'] = np.random.choice([0.01, 0.05, 0.1, 0.5], 1)[0]    #[0,1]
    param['max_depth'] = np.random.choice([4,5,6,7,8], 1)[0]                       #[1,]
    param['min_child_weight'] = np.random.choice([0,1,2], 1)[0]                    #[0,]
    param['subsample'] = np.random.choice([0.5,0.6,0.7,0.8], 1)[0]                 #(0,1]
    param['colsample_bytree'] = np.random.choice([0.5,0.6,0.7,0.8], 1)[0]          #(0,1]

    
    print 'params: %s' % param
    #init result for this round
    ensenble_training_result[i] = {}
    ensenble_training_result[i]['param'] = param
    
    cv_scores = []
    kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2016)
    fold = 0 
    for dev_index, val_index in kf.split(range(train_x.shape[0])):
            print "Fold: %s" % fold
            dev_x, val_x = train_mix_model[dev_index,:], train_mix_model[val_index,:]
            dev_y, val_y = train_y[dev_index], train_y[val_index]

            # Xbgoost parameter 
            val_y_pred, model = runXGB(param, dev_x, dev_y, val_x, val_y)
            cv_scores.append(log_loss(val_y, val_y_pred))

            print(cv_scores)
            fold += 1
    
    ensenble_training_result[i]['cv_scores'] = cv_scores

round: 0
params: {'num_class': 3, 'silent': 1, 'eval_metric': 'mlogloss', 'eta': 0.01, 'min_child_weight': 0, 'subsample': 0.80000000000000004, 'seed': 0, 'objective': 'multi:softprob', 'colsample_bytree': 0.80000000000000004, 'max_depth': 7}
Fold: 0
[0]	train-mlogloss:1.08378	test-mlogloss:1.0838
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 20 rounds.
[100]	train-mlogloss:0.332147	test-mlogloss:0.33278
[200]	train-mlogloss:0.116574	test-mlogloss:0.117731


KeyboardInterrupt: 

In [14]:
def get_submit_data(param, train_x, train_y, test_df, num_rounds=500):
    print 'param: {0}'.format(param)
    test_x = feature_preprocessing(test_df, False)
    preds, model = runXGB(param, train_x, train_y, test_x, num_rounds=num_rounds)
    out_df = pd.DataFrame(preds)
    out_df.columns = ["high", "medium", "low"]
    out_df["listing_id"] = test_df.listing_id.values
    out_df = out_df[["listing_id","high", "medium", "low" ]]
    print 'shape: {0}'.format(out_df.shape)
    return out_df

In [359]:

out_df = get_submit_data(param, train_x, train_y, test_df, num_rounds=2000)
out_df.to_csv("xgb_managerid.csv", index=False)


param: {'num_class': 3, 'silent': 1, 'eval_metric': 'mlogloss', 'eta': 0.1, 'min_child_weight': 0, 'subsample': 0.7, 'seed': 0, 'objective': 'multi:softprob', 'colsample_bytree': 0.7, 'max_depth': 6}
                 quadkey_15_bathrooms  quadkey_15_bedrooms  quadkey_15_price
quadkey_15                                                                  
021333333001101                   1.0                  0.0            2200.0
023010211023323                   1.0                  2.0            3200.0
023012311310010                   1.0                  1.0            2425.0
023101012323320                   1.0                  2.0            2150.0
023130121200020                   3.0                  3.0             500.0
tr_sparse(74659, 100)
tr_sparse_desc(74659, 100)
cloumns: Index([                u'bathrooms',                  u'bedrooms',
                        u'latitude',                 u'longitude',
                           u'price',                u'diff_price',
  

ValueError: feature_names mismatch: ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60', 'f61', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68', 'f69', 'f70', 'f71', 'f72', 'f73', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79', 'f80', 'f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90', 'f91', 'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99', 'f100', 'f101', 'f102', 'f103', 'f104', 'f105', 'f106', 'f107', 'f108', 'f109', 'f110', 'f111', 'f112', 'f113', 'f114', 'f115', 'f116', 'f117', 'f118', 'f119', 'f120', 'f121', 'f122', 'f123', 'f124', 'f125', 'f126', 'f127', 'f128', 'f129', 'f130', 'f131', 'f132', 'f133', '

(74659, 4)

In [133]:
## gen test_data model

In [20]:
# get preds by each model
mix_model_test_df = pd.DataFrame()
test_x = feature_preprocessing(test_df, False)
for i in xrange(50):
    print 'round: %s' % i
    param = {}
    # fixed parems
    param['objective'] = 'multi:softprob'
    param['eval_metric'] = "mlogloss"
    param['seed'] = 0
    param['silent'] = 1 
    param['num_class'] = 3
    
    # random
    param['eta'] = np.random.choice([0.01, 0.05, 0.1, 0.5, 1], 1)[0]    #[0,1]
    param['max_depth'] = np.random.choice([2,3,4,5,6,7,8,9,10], 1)[0]                       #[1,]
    param['min_child_weight'] = np.random.choice([0,1,2,3,4,5], 1)[0]                    #[0,]
    param['subsample'] = np.random.choice([0.4,0.5,0.6,0.7,0.8], 1)[0]                 #(0,1]
    param['colsample_bytree'] = np.random.choice([0.4,0.5,0.6,0.7,0.8], 1)[0]          #(0,1]
    print 'parma: %s' % param
    preds, model = runXGB(param, train_x, train_y, test_x, num_rounds=200)
    preds_df = pd.DataFrame(preds, columns=['pred_high_%s' % i, 'pred_medium_%s' % i, 'pred_low_%s' % i])
    
    if i == 0:
        mix_model_test_df = preds_df
    else:
        mix_model_test_df = pd.concat([ mix_model_test_df, preds_df], axis = 1)

    print 'dataframe: {0}'.format(mix_model_test_df.shape)

                 quadkey_15_bathrooms  quadkey_15_bedrooms  quadkey_15_price
quadkey_15                                                                  
021333333001101                   1.0                  0.0            2200.0
023010211023323                   1.0                  2.0            3200.0
023012311310010                   1.0                  1.0            2425.0
023101012323320                   1.0                  2.0            2150.0
023130121200020                   3.0                  3.0             500.0
tr_sparse(74659, 100)
tr_sparse_desc(74659, 100)
cloumns: Index([                u'bathrooms',                  u'bedrooms',
                        u'latitude',                 u'longitude',
                           u'price',                u'diff_price',
                  u'diff_bathrooms',             u'diff_bedrooms',
                     u'ratio_price',            u'building_price',
              u'building_bathrooms',         u'building_bedrooms',
 

In [16]:
mix_model_test_df.shape

(74659, 75)

In [148]:
preds, model = runXGB(param, mix_model_df, train_y, mix_model_test_df, num_rounds=2000)
out_df = pd.DataFrame(preds)
out_df.columns = ["high", "medium", "low"]
out_df["listing_id"] = test_df.listing_id.values
out_df = out_df[["listing_id","high", "medium", "low" ]]


In [149]:
out_df.to_csv("xgb_10_iter_ensemble.csv", index=False)

In [150]:
param

{'colsample_bytree': 0.80000000000000004,
 'eta': 0.01,
 'eval_metric': 'mlogloss',
 'max_depth': 7,
 'min_child_weight': 0,
 'num_class': 3,
 'objective': 'multi:softprob',
 'seed': 0,
 'silent': 1,
 'subsample': 0.80000000000000004}

In [25]:
mix_model_test_df.head()

NameError: name 'mix_model_test_df' is not defined

In [17]:
def gen_result_from_mixed_model(mixed_model_result, raw_test_df):
    mixed_model_result['high'] = mixed_model_result[['pred_high_'+str(i) for i in xrange(10)]].mean(axis=1)
    mixed_model_result['medium'] = mixed_model_result[['pred_medium_'+str(i) for i in xrange(10)]].mean(axis=1)
    mixed_model_result['low'] = mixed_model_result[['pred_low_'+str(i) for i in xrange(10)]].mean(axis=1)
    mixed_model_result["listing_id"] = raw_test_df.listing_id.values
    mixed_model_result = mixed_model_result[["listing_id","high", "medium", "low" ]]
    return mixed_model_result


In [21]:
result = gen_result_from_mixed_model(mix_model_test_df, test_df)

In [31]:
mix_model_df.shape

(49352, 33)

In [33]:
test_df.shape

(74659, 14)

In [181]:
temp['high'] = temp[['pred_high_'+str(i) for i in xrange(10)]].mean(axis=1)
temp['medium'] = temp[['pred_medium_'+str(i) for i in xrange(10)]].mean(axis=1)
temp['low'] = temp[['pred_low_'+str(i) for i in xrange(10)]].mean(axis=1)

In [182]:
temp["listing_id"] = test_df.listing_id.values
temp = temp[["listing_id","high", "medium", "low" ]]

In [40]:
result.head()

,listing_id,high,medium,low
0,7142618,0.072869,0.359702,0.567428
1,7210040,0.301329,0.429328,0.269343
2,7103890,0.010274,0.111844,0.877883
3,7143442,0.045526,0.272457,0.682017
4,6860601,0.010091,0.141232,0.848677


In [22]:
result.to_csv("xgb_50_iter_mean.csv", index=False)